In [1]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key="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",
    temperature=0,
)

response = llm.invoke("The first person to walk on the moon was?")
print(response.content)


The first person to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [12]:
from langchain_community.document_loaders import WebBaseLoader


loader = WebBaseLoader("https://careers.nike.com/lead-software-engineer-itc/job/R-51405")

page_data = loader.load().pop().page_content
print(page_data)



























Lead Software Engineer, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a L

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
   ### Scraped Text from website
   {page_data}
   
   ### Instructions
   The scraped text is from the career's page of a website. 
   Your job is to extract the job postings and returen them in a jSON format containing the following keys:
   'role', 'experience', 'skills' and 'description'
   only return the JSON object 
   ## VALID JSON (NO PREAMBLE)
   """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)







```json
{
  "role": "Lead Software Engineer, ITC",
  "experience": "7-10 years",
  "skills": [
    "UI Programming languages like React or Vue Js",
    "Server-side programming languages such as Node.js, Python",
    "Serverless architectures",
    "NoSQL databases like Dynamo DB",
    "RESTful APIs and web services",
    "Cloud platforms preferably AWS",
    "Git, GitHub, or similar version control tools",
    "Unit testing, integration testing, and test-driven development (TDD)"
  ],
  "description": "Design & Develop: Build, test, and maintain web applications and services across the full stack, providing end-to-end functionality. Lead Technical Initiatives: Lead and contribute to architectural decisions, technical strategy, and feature planning. Collaborate: Work closely with product managers, designers, stakeholders and other engineers to define requirements, technical specifications, and deliver quality software."
}
```


In [14]:

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer, ITC',
 'experience': '7-10 years',
 'skills': ['UI Programming languages like React or Vue Js',
  'Server-side programming languages such as Node.js, Python',
  'Serverless architectures',
  'NoSQL databases like Dynamo DB',
  'RESTful APIs and web services',
  'Cloud platforms preferably AWS',
  'Git, GitHub, or similar version control tools',
  'Unit testing, integration testing, and test-driven development (TDD)'],
 'description': 'Design & Develop: Build, test, and maintain web applications and services across the full stack, providing end-to-end functionality. Lead Technical Initiatives: Lead and contribute to architectural decisions, technical strategy, and feature planning. Collaborate: Work closely with product managers, designers, stakeholders and other engineers to define requirements, technical specifications, and deliver quality software.'}

In [17]:
print(type(json_res))

import pandas as pd
df= pd.read_csv('my_portfolio.csv')
df


<class 'dict'>


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [22]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

job = json_res



In [23]:

job

{'role': 'Lead Software Engineer, ITC',
 'experience': '7-10 years',
 'skills': ['UI Programming languages like React or Vue Js',
  'Server-side programming languages such as Node.js, Python',
  'Serverless architectures',
  'NoSQL databases like Dynamo DB',
  'RESTful APIs and web services',
  'Cloud platforms preferably AWS',
  'Git, GitHub, or similar version control tools',
  'Unit testing, integration testing, and test-driven development (TDD)'],
 'description': 'Design & Develop: Build, test, and maintain web applications and services across the full stack, providing end-to-end functionality. Lead Technical Initiatives: Lead and contribute to architectural decisions, technical strategy, and feature planning. Collaborate: Work closely with product managers, designers, stakeholders and other engineers to define requirements, technical specifications, and deliver quality software.'}

In [1]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Kunal, a machine learning engineer.  
        I’m a machine learning engineer with a strong background in data science and research,
        eager to contribute to in the industry's innovative work in machine learning.
        I’m excited about the possibility of bringing my skills to your team.
        With hands-on experience in designing and deploying machine learning models,
        My expertise spans tools like TensorFlow, PyTorch, and scikit-learn, alongside advanced techniques in 
        natural language processing, computer vision, and predictive analytics. 
        Using the above information, write a cold email to the client regarding the job mentioned above.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list} 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

NameError: name 'PromptTemplate' is not defined